In [10]:
# xmlrpc_server.ipynb
import copy
from xmlrpc.server import *
from xmlrpc.client import *
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8010), 
                            requestHandler=RequestHandler)

stats_server = ServerProxy("http://localhost:8011")


def slice_log(action_names=[], 
              action_start_date=datetime.datetime(1, 1, 1, 0, 0).strftime("%Y-%m-%d %H:%M:%S"), 
              action_stop_date=datetime.datetime(9999, 12, 31, 23, 59, 59, 999999).strftime("%Y-%m-%d %H:%M:%S"),
              s_duration=0.0,
              e_duration=float('inf')):
    try:
        df = pd.read_json(stats_server.get_log_df())
        print(df)
        if len(action_names) != 0:
            df = df.loc[df['action_name'].isin(action_names)]
        df = df.loc[df['date'] >= action_start_date]
        df = df.loc[df['date'] <= action_stop_date]
        df = df.loc[df['duration'] >= s_duration]
        df = df.loc[df['duration'] <= e_duration]
        return df.to_json()
    except Exception as e:
        print("WARWING! stats server error!")
        print(e)
        return "{}"
        
server.register_function(slice_log, 'slice_log')

# Тест
def ping():
    
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
    
server.register_function(black_list_check, 'black_list_check')
    
def black_list_check_with_date_and_fullname(sname, nname, pname, date):
    
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any((frame['Surname'] + frame['Name']  + frame['Patronym']  + frame['Birth']) == sname + nname + pname + date)
    print(exist)
    if (exist):
        return sname + nname + pname + " with birthday: " + date + " : "+ "bad_boy"
    else:
        return sname + nname + pname + " with birthday: " + date + " : "+ "good_boy"

server.register_function(black_list_check_with_date_and_fullname, 'black_list_check_with_date_and_fullname')

# Бинарная передача данных
def send_back_binary(bin_data):
    
    data = bin_data.data
    return Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    isColored = len(img_arr.shape) == 3
    for i in range(height):
        for j in range(width):
            if isColored:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


def binary(val, border):
    if val < border:
        return 0
    else:
        return 255


# Бинаризация изображения по задаваемому порогу 1-255
def binary_img(bin_data, border):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            try:
                img_arr[i][j][0] = binary(img_arr[i][j][0], border)
                img_arr[i][j][1] = binary(img_arr[i][j][1], border)
                img_arr[i][j][2] = binary(img_arr[i][j][2], border)
            except Exception:
                img_arr[i][j] = binary(img_arr[i][j], border)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(binary_img, 'binary_img')

# Поворот относительно вертикали
def reverse(bin_data):
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width // 2):
            temp = copy.deepcopy(img_arr[i][j])
            img_arr[i][j] = copy.deepcopy(img_arr[i][width - j - 1])
            img_arr[i][width - j - 1] = copy.deepcopy(temp)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(reverse, 'reverse')

print("Listening on " + str(server.server_address))

try: 
    server.serve_forever()
    print('That was a nice nap.')
except KeyboardInterrupt:
    server.server_close()
    print('What a rude awakening!')


Listening on ('127.0.0.1', 8010)


C:\Users\pavel\AppData\Local\Temp\ipykernel_8072\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [28/Nov/2023 00:25:12] "POST /RPC2 HTTP/1.1" 200 -


    id action_name                date     duration
0    1         pow 2023-11-28 00:03:41  2038.251638
1    2        ping 2023-11-28 00:04:29  2015.470028
2    3         now 2023-11-28 00:04:35  2018.880129
3    4        type 2023-11-28 00:04:41  2028.809547
4    5        type 2023-11-28 00:04:47  2028.699398
5    6        type 2023-11-28 00:04:53  2036.431313
6    7        type 2023-11-28 00:04:59  2016.364098
7    8        type 2023-11-28 00:05:05  2028.347492
8    9        type 2023-11-28 00:05:12  2057.499886
9   10        type 2023-11-28 00:05:18  2017.121077
10  11         sum 2023-11-28 00:05:24  2009.181261
11  12         pow 2023-11-28 00:05:30  2023.413420
12  13   slice_log 2023-11-28 00:05:38  4043.475389
13  14   slice_log 2023-11-28 00:06:29  4072.201967
14  15   slice_log 2023-11-28 00:07:32  4077.429295
15  16   slice_log 2023-11-28 00:09:01  4103.732109
16  17   slice_log 2023-11-28 00:10:23  4058.773994
17  18   slice_log 2023-11-28 00:11:31  4032.645702
18  19   sli

C:\Users\pavel\AppData\Local\Temp\ipykernel_8072\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [28/Nov/2023 00:25:51] "POST /RPC2 HTTP/1.1" 200 -


    id action_name                date     duration
0    1         pow 2023-11-28 00:03:41  2038.251638
1    2        ping 2023-11-28 00:04:29  2015.470028
2    3         now 2023-11-28 00:04:35  2018.880129
3    4        type 2023-11-28 00:04:41  2028.809547
4    5        type 2023-11-28 00:04:47  2028.699398
5    6        type 2023-11-28 00:04:53  2036.431313
6    7        type 2023-11-28 00:04:59  2016.364098
7    8        type 2023-11-28 00:05:05  2028.347492
8    9        type 2023-11-28 00:05:12  2057.499886
9   10        type 2023-11-28 00:05:18  2017.121077
10  11         sum 2023-11-28 00:05:24  2009.181261
11  12         pow 2023-11-28 00:05:30  2023.413420
12  13   slice_log 2023-11-28 00:05:38  4043.475389
13  14   slice_log 2023-11-28 00:06:29  4072.201967
14  15   slice_log 2023-11-28 00:07:32  4077.429295
15  16   slice_log 2023-11-28 00:09:01  4103.732109
16  17   slice_log 2023-11-28 00:10:23  4058.773994
17  18   slice_log 2023-11-28 00:11:31  4032.645702
18  19   sli

C:\Users\pavel\AppData\Local\Temp\ipykernel_8072\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [28/Nov/2023 00:26:21] "POST /RPC2 HTTP/1.1" 200 -


    id action_name                date     duration
0    1         pow 2023-11-28 00:03:41  2038.251638
1    2        ping 2023-11-28 00:04:29  2015.470028
2    3         now 2023-11-28 00:04:35  2018.880129
3    4        type 2023-11-28 00:04:41  2028.809547
4    5        type 2023-11-28 00:04:47  2028.699398
5    6        type 2023-11-28 00:04:53  2036.431313
6    7        type 2023-11-28 00:04:59  2016.364098
7    8        type 2023-11-28 00:05:05  2028.347492
8    9        type 2023-11-28 00:05:12  2057.499886
9   10        type 2023-11-28 00:05:18  2017.121077
10  11         sum 2023-11-28 00:05:24  2009.181261
11  12         pow 2023-11-28 00:05:30  2023.413420
12  13   slice_log 2023-11-28 00:05:38  4043.475389
13  14   slice_log 2023-11-28 00:06:29  4072.201967
14  15   slice_log 2023-11-28 00:07:32  4077.429295
15  16   slice_log 2023-11-28 00:09:01  4103.732109
16  17   slice_log 2023-11-28 00:10:23  4058.773994
17  18   slice_log 2023-11-28 00:11:31  4032.645702
18  19   sli

C:\Users\pavel\AppData\Local\Temp\ipykernel_8072\64881279.py:25: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df())
127.0.0.1 - - [28/Nov/2023 00:26:59] "POST /RPC2 HTTP/1.1" 200 -


    id action_name                date     duration
0    1         pow 2023-11-28 00:03:41  2038.251638
1    2        ping 2023-11-28 00:04:29  2015.470028
2    3         now 2023-11-28 00:04:35  2018.880129
3    4        type 2023-11-28 00:04:41  2028.809547
4    5        type 2023-11-28 00:04:47  2028.699398
5    6        type 2023-11-28 00:04:53  2036.431313
6    7        type 2023-11-28 00:04:59  2016.364098
7    8        type 2023-11-28 00:05:05  2028.347492
8    9        type 2023-11-28 00:05:12  2057.499886
9   10        type 2023-11-28 00:05:18  2017.121077
10  11         sum 2023-11-28 00:05:24  2009.181261
11  12         pow 2023-11-28 00:05:30  2023.413420
12  13   slice_log 2023-11-28 00:05:38  4043.475389
13  14   slice_log 2023-11-28 00:06:29  4072.201967
14  15   slice_log 2023-11-28 00:07:32  4077.429295
15  16   slice_log 2023-11-28 00:09:01  4103.732109
16  17   slice_log 2023-11-28 00:10:23  4058.773994
17  18   slice_log 2023-11-28 00:11:31  4032.645702
18  19   sli